# Analysis of 4D-STEM data of clusters in Al alloys

This notebook handles 4D-STEM data and was originally written to investigate scanning precession electron diffraction (SPED) data from an Al-Zn-Mg alloy containing clusters. The clusters are 1-2 nm spherical particles completely embedded in the Al host material. They have their own crystal structure and the notebook aims at extracting PED patterns for single clusters. Various methods as calibrating the data, creating masks in real- and reciprocal space, creating a simplified dataset where each pixel containing a cluster is the average of all the PED patterns in that particular pattern and decomposition of the dataset are available. The code runs on pyxem v. 13.2.2. 

The notebook was originally written by dr. Emil Christiansen (emichr) but was rewritten by Elisabeth Thronsen to fit the purpose of analysing SPED data from clusters in Al alloys.

The notebook contains the following sections:

    [1] Loading of data and calibration
    [2] Create Virtual BF/DF
    [3] Create masks
    [4] Create simplified dataset
    [5] Decomposition


In [1]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.ion()
import numpy as np
from scipy import sqrt
import hyperspy.api as hs
import pyxem as px
from skimage.feature import blob_log
import os

directory = r'E:\PhD\MERLIN\ZM42\Na4d_AA8min\2021_01_22_s10a\20210122 105644'.replace('\\', '\\\\') # Directory where you have the data
os.chdir(r'C:\Users\elisathr\OneDrive - NTNU\PhD\ALLDESIGN\Studying_clustering_electron_diffraction'.replace('\\', '\\\\')) # Directory with ClusterAnalysis
import ClusterAnalysis as ca 
os.chdir(directory) # Change back to directory where you have the files.

from ClusterAnalysis import VDF_mask_methods as vdfm
from skimage import (
    color, feature, filters, io, measure, morphology, segmentation, util
)

## Load data
Load data first as a .mib-file before converting it to a .hspy file.

In [2]:
# Define the material:

material = 'Al'
a = 4.05
b = a
c = a
alpha = 90.
beta=alpha
gamma=alpha

In [3]:
d_max = (2, 2, 0) #maximum reflection to include when cropping reciprocal space

In [4]:
# Define the file name of the data that you want to open
file_name = '/Scan2_SPED0.3_exp30ms_spot0.5_step0.3_alpha4_100x100.hspy'

In [5]:
'''
Load data. This routine will open data based on the file extension. Accepted files are .mib, .hspy and .hdf5. If loading a .mib-file, the scan size and detector size need to be inputted with the nx, ny
and dx, dy parameters. Upon loading a .mib-file, the routine will save the dataset as a .hspy-file.
'''

load_masks = False # Set to True if importing binary masks from file
if '.mib' in file_name:
    signal = px.load_mib(directory + '/' + file_name)
    nx, ny = 100, 100
    dx, dy = 256, 256 #Detector pixels
    data_array = signal.data.reshape((nx, ny, dx, dy)) #Assume data size!
    data_array = data_array.rechunk((32, 32, 32, 32)) #Rechunk data
    signal = px.signals.LazyElectronDiffraction2D(data_array)
    try:
        with open((directory + '/' + file_name).replace('.mib', '.hdr'), 'r') as hdr:
            header = ''.join(hdr.readlines())
            print(header)
            signal.original_metadata['Header'] = header
    except:
        print('Header from .mib file not read and imported to metadata')
    signal.save(directory + '/' + file_name.replace(".mib", ".hspy"), chunks=(32, 32, 32, 32), overwrite=True)
    signal = hs.load(directory + '/' + file_name.replace(".mib", ".hspy"))
    signal = px.signals.ElectronDiffraction2D(signal)
else:
    try:
        signal = hs.load(directory + '\\\\' + file_name)
        print('Sucesfully loaded dataset.')
    except:
        print('Could not load the data with hs.load')
    if load_masks:
        try:
            vdf = hs.load(directory + '/' + file_name.replace('.hspy', '_vdf.hdf5'))
            print('Sucesfully loaded vdf.')
        except:
            print('Could not load vdf')
        try:
            vdf_mask = hs.load(directory + '/' + file_name.replace('.hspy', '_vdf_mask.hdf5'))
            print('Sucesfully loaded vdf mask.')
        except:
            print('Could not load vdf mask')
        try:
            diff_mask = hs.load(directory + '/' + file_name.replace('.hspy', '_diff_mask.hdf5'))
            print('Sucesfully loaded diff mask.')
        except:
            print('Could not load diff mask')

Sucesfully loaded dataset.


### Prepare data dimensions and calibrate

#### Calibrate the data in reciprocal space

Here we use the Line2DROI object from hyperspy to get a profile plot over diffraction reflections we know the distance between.

In [7]:
signal.plot()

In [6]:
# Plot the data and find the center of the central beam

plt.close('all')
signal.plot()

In [10]:
center = (128, 128)
#center = None

In [9]:
# Use the DPs to set the scale in recirpocal space
try:
    DP_scale = signal.metadata.Signal.calibration.diffraction
    calibration_done = True
except AttributeError:
    calibration_done = False
    plt.close("all")
    signal.plot()
    lin = hs.roi.Line2DROI(x1=20, y1=20, x2=40, y2=40, linewidth=5)
    lin.add_widget(signal, axes=signal.axes_manager.signal_axes)

In [10]:
if not calibration_done:
    trace = lin(signal, axes=signal.axes_manager.signal_axes)
    trace.sum((0,1)).plot()

In [11]:
x_1 = 15 #Pos of spot -200
x_2 = 144 # Pos of spot 200
hkl1=np.array([2,0,0])
hkl2=np.array([-2,0,0])
hkl=hkl1-hkl2
precision = 3 #Number of decimals for calibration
d_px = np.abs(x_1-x_2) #px
if c is a and b is a:
    if beta is alpha and gamma is alpha:
        d_nm = np.sqrt((hkl[0]**2 + hkl[1]**2 + hkl[2]**2)/a**2) #1/Å
DP_scale = round(d_nm/d_px, precision) #1/Å / px
#DP_scale = 1
if center is not None:
    signal.set_diffraction_calibration(DP_scale, center=(center[0]*DP_scale, center[1]*DP_scale))
else:
    signal.set_diffraction_calibration(DP_scale)
print('Theoretical d-spacing of {material} ({h:.0f} {k:.0f} {l:.0f}): {d_nm} 1/Å\nDistance between {material} ({h1:.0f} {k1:.0f} {l1:.0f}) and ({h2:.0f} {k2:.0f} {l2:.0f}) in DP: {d_px} px\nDiffraction pattern calibration: {scale} 1/Å / px'.format(material = material, h=hkl[0], k=hkl[1], l=hkl[2], d_nm=round(d_nm, precision), h1=hkl1[0], k1=hkl1[1], l1=hkl1[2], h2=hkl2[0], k2=hkl2[1], l2=hkl2[2], d_px=round(d_px, precision), scale = round(DP_scale, precision)))
signal.metadata.add_dictionary({'Signal':{'calibration':{'diffraction':DP_scale}}})
calibration_done = True

Theoretical d-spacing of Al (4 0 0): 0.988 1/Å
Distance between Al (2 0 0) and (-2 0 0) in DP: 129 px
Diffraction pattern calibration: 0.008 1/Å / px


In [12]:
# Check that the axes make sense:
plt.close('all')
signal.axes_manager

<Axes manager, axes: (100, 100|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |    100 |     25 |       0 |       1 | <undefined> 
     <undefined> |    100 |     38 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |      -1 |   0.008 | $A^{-1}$ 
              ky |    256 |        |      -1 |   0.008 | $A^{-1}$

#### Calibrate in real space and set the units and axes in both reciprocal- and real space

In [13]:
signal.axes_manager[0].offset = 0 # x-axis
signal.axes_manager[1].offset = 0 # y-axis
SX, SY = .462, .462 # The scan step size
  
signal.axes_manager[0].scale = SX#
signal.axes_manager[1].scale = SY#
signal.metadata.add_dictionary({'Signal':{'calibration':{'position':signal.axes_manager[0].scale}}})

signal.axes_manager[0].units = 'nm' 
signal.axes_manager[1].units = 'nm' 

signal.axes_manager[2].units = 'Å^-1'
signal.axes_manager[3].units = 'Å^-1'

signal.axes_manager[0].name = 'x'
signal.axes_manager[1].name = 'y'
signal.axes_manager[2].name = 'dx'
signal.axes_manager[3].name = 'dy'

signal.axes_manager

<Axes manager, axes: (100, 100|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    100 |     25 |       0 |    0.46 |     nm 
               y |    100 |     38 |       0 |    0.46 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |    256 |        |      -1 |   0.008 |   Å^-1 
              dy |    256 |        |      -1 |   0.008 |   Å^-1

## Create Virtual BF/DF

Create a virtual BD/DF image by using the interactive ROI in hyperspy. The resultant virtual image should contain all features of interest and will be binarized at a later stage when the real space mask is created.

In [11]:
try:
    DP_scale
except:
    DP_scale = signal.axes_manager[2].scale
roi = hs.roi.CircleROI(cx=center[0]*DP_scale/256,cy=center[1]*DP_scale/256, r_inner=0*DP_scale, r=30*DP_scale)
plt.close('all')
signal.plot_integrated_intensity(roi=roi)

In [15]:
# Save the virtual image created with the interactive aperture:
vdf = signal.get_integrated_intensity(roi)

In [12]:
signal.metadata

├── General
│   ├── direct_detector = False
│   └── title = 7M42_NA4days_AA8min_SPED_scan2
├── Masks
│   ├── DP
│   │   ├── inverted = <Signal2D, title: Al Mask (inverted), dimensions: (|256, 256)>
│   │   └── normal = <Signal2D, title: Al Mask, dimensions: (|256, 256)>
│   ├── Parameters
│   │   ├── Diffraction
│   │   │   ├── blob
│   │   │   │   ├── max_sigma = 40
│   │   │   │   ├── min_sigma = 11
│   │   │   │   ├── num_sigma = 1
│   │   │   │   ├── overlap = 0
│   │   │   │   └── threshold = 0.03
│   │   │   ├── r
│   │   │   │   ├── maximum = 6
│   │   │   │   └── minimum = 4
│   │   │   ├── x = 0
│   │   │   └── y = 0
│   │   └── VDF
│   │       └── Type = Binary_vdf_mask
│   └── VDF
│       ├── inverted = <Signal2D, title: , dimensions: (|100, 100)>
│       └── normal = <Signal2D, title: VDF mask, dimensions: (|100, 100)>
├── Sample
│   ├── alloy = 7M42
│   ├── deformation = SPED
│   └── temper = NA4days_AA8min
├── Signal
│   ├── binned = False
│   ├── calibration
│   │   ├── diffraction = 0.008
│   │   └── position = 0.3
│   └── signal_type = electron_diffraction
└── VDF_interactive
    ├── data = <Signal2D, title: VDF, dimensions: (|100, 100)>
    └── title = VDF

## Create Masks

### Reciprocal mask

In [6]:
'''
Since we are not interested in the Al in this case, we create a mask in reciprocal space masking out the Al reflections. The mask can be prepared from the average DP from the full data stack if the 
cluster reflections are not too prominent. If however the cluster spots are prominent in the average DP, use a single DP where no clusters are present defined by the isig_x and isig_y parameters. 
The blob parameters for the blob detection algorithm are afound here: https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_blob.html.
First we check if blobs are detected in the metadata, if it is not, we have to create the mask from scratch.
'''

try:
    isig_x = signal.metadata.Masks.Parameters.Diffraction.x
    isig_y = signal.metadata.Masks.Parameters.Diffraction.y
    blob_params = signal.metadata.Masks.Parameters.Diffraction.blob
    minimum_r = signal.metadata.Masks.Parameters.Diffraction.r.minimum
    maximum_r = signal.metadata.Masks.Parameters.Diffraction.r.maximum
    print('Loaded blob-finding parameters from signal metadata')
    new_blobs = False
except AttributeError:
    print('No blob-finding metadata was found. Plot signal and find a pure Al pattern.')
    signal.plot()
    new_blobs = True

Loaded blob-finding parameters from signal metadata


If no blob-parameters are found, enter parameters manually (isig_x and isig_y are coordinates in signal space where you have pure Al). Change the blob_params to mask out only the Al reflections in the code under.

In [18]:
new_blobs = True
plt.close('all')
if new_blobs:
    isig_x = 24
    isig_y = 25
    blob_params = blob_params = {'min_sigma': 0.001, 'max_sigma':40, 'num_sigma': 1, 'overlap': 0, 'threshold': 0.03}

Search for blobs in pattern

In [128]:
signal.inav[isig_x, isig_y].plot()

In [14]:
plt.close('all')
from numpy.lib.scimath import sqrt
from skimage import exposure

# First we define which DP to use for creating the binary mask, either the mean DP or one single DP defined by isig_x and isig_y. Uncomment the one you don't want to use under and adjust gamma accordingly

#diffraction_pattern = signal.inav[isig_x,isig_y]
diffraction_pattern = signal.mean((0,1)) + 1 # To avoid issues with dead pixels
diffraction_pattern = exposure.adjust_gamma(diffraction_pattern.data, 0.2)

mask_xsize = signal.axes_manager.shape[2]
mask_ysize = signal.axes_manager.shape[3]

#Find Blobs
blobs = blob_log(diffraction_pattern, min_sigma=blob_params['min_sigma'], max_sigma=blob_params['max_sigma'], num_sigma = blob_params['num_sigma'], overlap = blob_params['overlap'], threshold = blob_params['threshold'])#, threshold=200.0)
blobs[:, 2] = blobs[:, 2] * sqrt(2) #scale blob size to "look" like a radii.

#Make figure to show blobs and mask side-by-side
fig, [orig_ax, mask_ax] = plt.subplots(1, 2)

#Show original image in one axis
orig_ax.imshow(diffraction_pattern, interpolation='nearest')

#Initialize mask array as an array with False everywhere
mask_array = np.zeros((mask_ysize, mask_xsize), dtype = bool)

#Loop over all blobs found by the Laplacian of Gaussian algorithm above, and mask out each blob. Blobs with very small or large radii will be given "minimum_r" or "maximum_r" as radii instead.
for k, blob in enumerate(blobs):
    y, x, r = blob #x, y, and radii of current blob
    print('Now x = {:.0f}, y = {:.0f}, r = {:.3f} (no: {})'.format(x, y, r, k))
    r += 8 #Possibly add to radii to increase size slightly

    # The mask can be shifted for specific relfections
    if k == 4:
        x += 0
        y += 0
        r += 5     
              
    #Make a plt.circle object to show the position of the mask in the original diffraction pattern and add it as a patch
    c = plt.Circle((x, y), r, color='red', linewidth=1, fill=False) 
    orig_ax.add_patch(c)
    
    #Create a mask using np.ogrid
    mask_x, mask_y = np.ogrid[-y:mask_ysize-y, -x:mask_xsize-x]
    mask_spot = mask_x**2 + mask_y**2 <= r**2
    mask_array[mask_spot] = True

orig_ax.set_axis_off()
mask_ax.set_axis_off()

#Show the mask next to the original diffraction pattern
mask_ax.imshow(mask_array)
plt.tight_layout()
plt.show()

#Mask out Al-reflections in the dataset:

Al_mask = hs.signals.Signal2D(mask_array)
Al_mask.metadata.General.title = 'Al Mask'
Al_mask_inverted = ~ Al_mask

Al_mask_inverted.metadata.General.title = 'Al Mask (inverted)'



Now x = 124, y = 192, r = 15.556 (no: 0)
Now x = 65, y = 122, r = 15.556 (no: 1)
Now x = 135, y = 63, r = 15.556 (no: 2)
Now x = 194, y = 132, r = 15.556 (no: 3)
Now x = 130, y = 127, r = 15.556 (no: 4)
Now x = 60, y = 187, r = 15.556 (no: 5)
Now x = 189, y = 197, r = 15.556 (no: 6)
Now x = 70, y = 58, r = 15.556 (no: 7)
Now x = 200, y = 67, r = 15.556 (no: 8)
Now x = 0, y = 117, r = 15.556 (no: 9)
Now x = 54, y = 255, r = 15.556 (no: 10)
Now x = 0, y = 52, r = 15.556 (no: 11)
Now x = 255, y = 203, r = 15.556 (no: 12)
Now x = 206, y = 0, r = 15.556 (no: 13)
Now x = 119, y = 255, r = 15.556 (no: 14)
Now x = 141, y = 0, r = 15.556 (no: 15)
Now x = 255, y = 138, r = 15.556 (no: 16)
Now x = 0, y = 183, r = 15.556 (no: 17)
Now x = 74, y = 0, r = 15.556 (no: 18)
Now x = 188, y = 255, r = 15.556 (no: 19)


In [16]:
# A circular mask can also be superimposed:
cutoff = 0.7 #None
X, Y = np.meshgrid(signal.axes_manager['dx'].axis, signal.axes_manager['dy'].axis)
mask_temp = ~(np.sqrt(X**2+Y**2) <= cutoff)

circular_mask = Al_mask_inverted * ~mask_temp

plt.imshow(circular_mask)

#### Store mask in metadata

In [17]:
signal.metadata.add_dictionary({'Masks':{'DP':{'normal': Al_mask}}})
signal.metadata.add_dictionary({'Masks':{'DP':{'inverted': Al_mask_inverted}}})
signal.metadata.add_dictionary({'Masks':{'Parameters':{'Diffraction':{'x':isig_x, 'y': isig_y, 'blob':blob_params, 'r': {'minimum': minimum_r, 'maximum': maximum_r}}}}})
signal.metadata.add_dictionary({'Masks':{'DP':{'circular': circular_mask}}})

In [18]:
# The circular mask can also be used to prepare a VDF with the aperture being determined by the mask:
roi = hs.roi.RectangularROI(left=0, top=0, right=253, bottom=252)
(px.signals.ElectronDiffraction2D((circular_mask*signal))).plot_integrated_intensity(roi=roi)

In [20]:
# Save the VDF based on the ciruclar mask:
vdf_circular_mask = (px.signals.ElectronDiffraction2D((circular_mask*signal))).get_integrated_intensity(roi=roi)
vdf_circular_mask.plot()

### Real space mask

To create a mask in real space, the following steps are executed: 
<ol>
<li>     Rolling ball correction to account for unenven background due to contamination build-up during acquisition </li>
<li>     Background subtraction using filtering regional maxima </li>
<li>     Apply a white top hat to the image outputted in 2 </li>
<li>     Threshold the image outputted in 3 by choosing a suitable threshold.  </li>
    </ol>

#### Define which VDF to threshold

In [20]:
plt.close('all')
vdf.plot()
vdf_circular_mask.plot()

In [21]:
#vdf = signal.metadata.VDF_interactive.data
vdf = vdf_circular_mask

#### Rolling ball correction

In [22]:
from skimage.filters import threshold_local
background = threshold_local(vdf.data, 49, offset=np.percentile(vdf,1), method='median')
bg_corrected = vdf.data - background

In [23]:
bg_corrected =  hs.signals.Signal2D(bg_corrected)

In [24]:
bg_corrected.axes_manager[0].scale=0.3
bg_corrected.axes_manager[1].scale = 0.3
bg_corrected.axes_manager[0].units='nm'
bg_corrected.axes_manager[1].units = 'nm'

In [25]:
# Compare the corrected and the raw image:
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].imshow(vdf)
ax[0].set_title('Raw VDF')
ax[1].imshow((bg_corrected))
ax[1].set_title('VDF corrected for uneven background')

Text(0.5, 1.0, 'VDF corrected for uneven background')

In [30]:
bg_corrected.plot()

#### Background subtraction

In [27]:
plt.close('all')
threshold = 10000 # Threshold for background subtraction
sigma = 0.5
#VDF_background_subtracted = filterLocalMaxima(2*vdf.data, threshold, sigma=2)
VDF_background_subtracted = vdfm.filterLocalMaxima(2*bg_corrected.data, threshold, sigma)
fig,ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].imshow(VDF_background_subtracted)
ax[1].imshow(bg_corrected)

#### White tophat

In [28]:
radius = 1.7 # Remove all bright spots under the radius
selem = morphology.disk(radius)
res = morphology.white_tophat(VDF_background_subtracted, selem)
fig,ax=plt.subplots(ncols=3, figsize=(20,8))
ax[0].imshow(bg_corrected)
ax[1].imshow(VDF_background_subtracted-res)
ax[2].imshow(VDF_background_subtracted)

#### Create binary image

In [29]:
threshold = 900
binary_vdf = (VDF_background_subtracted-res) > threshold
#binary_vdf = vdf.data > threshold
fig, ax = plt.subplots(ncols=3, figsize=(10,5))
ax[0].imshow(bg_corrected)
ax[1].imshow((VDF_background_subtracted-res))
ax[2].imshow(binary_vdf)
plt.show()
#plt.imshow(vdf*~binary_vdf)

In [30]:
Mask = hs.signals.Signal2D(binary_vdf) # Save as Signal2D

In [36]:
Mask.plot()

In [31]:
Mask.axes_manager = vdf.axes_manager

#### Edit mask manually

In [25]:
temp_mask = Mask.deepcopy()
temp_binary_vdf = np.copy(binary_vdf)

In [36]:
vdf.plot()

In [26]:
reg = hs.roi.RectangularROI(left=5., right=10., top=5., bottom=10.)
fig,ax = plt.subplots(ncols=4)
ax[0].imshow(temp_mask)
ax[1].imshow(VDF_background_subtracted-res)
ax[2].imshow(vdf)
ax[3].imshow(binary_vdf)
temp_mask.plot()

reg.add_widget(temp_mask)
mask_xsize = temp_mask.axes_manager.shape[0]
mask_ysize = temp_mask.axes_manager.shape[1]

In [58]:
start_x = int(reg.x)
start_y = int(reg.y)
stop_x = int(start_x + reg.width)
stop_y = int(start_y + reg.height)

In [59]:
temp_mask.isig[start_x:stop_x, start_y:stop_y] = False

In [60]:
plt.close('all')
fig,ax = plt.subplots(ncols=3)
ax[1].imshow(temp_mask)
ax[1].set_title('New mask')
ax[0].imshow(Mask)
ax[0].set_title('Original mask')
ax[2].imshow(vdf)
ax[2].set_title('VDF')

Text(0.5, 1.0, 'VDF')

When you're happy with the mask, save it:

In [61]:
Mask = temp_mask.deepcopy()

In [62]:
Mask.axes_manager = vdf.axes_manager

In [67]:
Mask.plot(scalebar = False)

### Save masks

In [32]:
mask_type = 'Binary_vdf_mask'
Mask_inverted = Mask.deepcopy()
Mask_inverted.data = ~Mask_inverted.data
signal.metadata.add_dictionary({'Masks':{'VDF':{'normal': Mask}}})
signal.metadata.add_dictionary({'Masks':{'VDF':{'inverted': Mask_inverted}}})
signal.metadata.add_dictionary({'Masks':{'Parameters':{'VDF':{'Type':mask_type}}}})
#signal.metadata.add_dictionary({'Cluster_mean_signal':cluster_mean_signal})
Mask.metadata.General.title = 'VDF mask'
Mask.axes_manager[0].scale = signal.axes_manager[0].scale
Mask.axes_manager[1].scale = signal.axes_manager[1].scale
Mask.axes_manager[0].units = signal.axes_manager[0].units
Mask.axes_manager[1].units = signal.axes_manager[1].units
signal.metadata.Masks.VDF.normal = Mask
signal.metadata.Masks.VDF.inverted = Mask_inverted

In [33]:
signal.metadata

├── General
│   ├── direct_detector = False
│   └── title = 7M42_NA4days_AA8min_SPED_scan2
├── Masks
│   ├── DP
│   │   ├── circular = <Signal2D, title: Al Mask (inverted), dimensions: (|256, 256)>
│   │   ├── inverted = <Signal2D, title: Al Mask (inverted), dimensions: (|256, 256)>
│   │   └── normal = <Signal2D, title: Al Mask, dimensions: (|256, 256)>
│   ├── Parameters
│   │   ├── Diffraction
│   │   │   ├── blob
│   │   │   │   ├── max_sigma = 40
│   │   │   │   ├── min_sigma = 11
│   │   │   │   ├── num_sigma = 1
│   │   │   │   ├── overlap = 0
│   │   │   │   └── threshold = 0.03
│   │   │   ├── r
│   │   │   │   ├── maximum = 6
│   │   │   │   └── minimum = 4
│   │   │   ├── x = 0
│   │   │   └── y = 0
│   │   └── VDF
│   │       └── Type = Binary_vdf_mask
│   └── VDF
│       ├── inverted = <Signal2D, title: , dimensions: (|100, 100)>
│       └── normal = <Signal2D, title: VDF mask, dimensions: (|100, 100)>
├── Sample
│   ├── alloy = 7M42
│   ├── deformation = SPED
│   └── temper = NA4days_AA8min
├── Signal
│   ├── binned = False
│   ├── calibration
│   │   ├── diffraction = 0.008
│   │   └── position = 0.3
│   └── signal_type = electron_diffraction
└── VDF_interactive
    ├── data = <Signal2D, title: VDF, dimensions: (|100, 100)>
    └── title = VDF

In [ ]:
signal.save(directory + file_name)

## Create simplified dataset where each pixel within a cluster will be the sum of DPs in that cluster

In [34]:
labeled = morphology.label(Mask.data, connectivity=2)
print('Number of individual clusters: ', len(np.unique(labeled[1:])))

Number of individual clusters:  35


In [35]:
clusters = []
import math
cluster_mean_signal = signal.deepcopy()
index = 0
for i in np.unique(labeled)[1:]:
    cluster = signal.data[labeled==i]
    cluster = px.signals.ElectronDiffraction2D(cluster)
    clusters.append(cluster)
    cluster_mean_signal.data[labeled==i] = cluster.sum((0))

In [36]:
cluster_mean_signal.plot()

### Plot and save average DPs from individual clusters

To enhance the signal in each average diffraction pattern, the following pixel-by-pixel transformation was applied to each pattern: \
    $$ I_{enhanced} = 1 - e^{\frac{I_{initial}}{n}} $$
Firstly, test with different n's to find an appropriate value for the data:

In [37]:
vdf.axes_manager[0].scale = 0.3
vdf.axes_manager[1].scale = 0.3
vdf.axes_manager[0].units = 'nm'
vdf.axes_manager[1].units = 'nm'

In [95]:
vdf.plot()

In [81]:
Mask.plot()

In [12]:
n = 98
from matplotlib import colors
cluster_now = clusters[0].max(0).data
test_dp = 1-np.exp((-cluster_now/n))
fig,ax = plt.subplots(ncols=2)
ax[0].imshow(test_dp)
ax[0].set_title('Filtered DP')
ax[1].imshow(cluster_now, norm=colors.LogNorm())
ax[1].set_title('Original DP (log)')

Text(0.5, 1.0, 'Original DP (log)')

In [59]:
# Compare the sum, mean, variance, std and max for a particular average DP:

from matplotlib import colors
cluster_now = clusters[0]
fig,ax = plt.subplots(ncols=5)
ax[0].imshow(cluster_now.sum(0), norm=colors.LogNorm())
ax[0].set_title('Sum')
ax[1].imshow(cluster_now.mean(0), norm=colors.LogNorm())
ax[1].set_title('Mean')
ax[2].imshow(cluster_now.var(0), norm=colors.LogNorm())
ax[2].set_title('Variance')
ax[3].imshow(cluster_now.std(0), norm=colors.LogNorm())
ax[3].set_title('Std')
ax[4].imshow(cluster_now.max(0), norm=colors.LogNorm())
ax[4].set_title('Max')


Text(0.5, 1.0, 'Max')

When you are happy with the enhanced image, save each average DP as a png. This routine makes it possible to save the mean DP and the raw DP, with logarithmic scale or with the inverted gaussian filter

In [104]:
plt.imshow(1-np.exp((-DP_sum.mean(0).data)/n_mean), cmap='gray')

In [106]:
counter = 0
n_mean = 15
n_raw = 100
output_directory = directory
from matplotlib import colors
for DP_sum in clusters:
    # Mean inverted gauss:
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.imshow(1-np.exp((-DP_sum.mean(0).data)/n_mean), cmap='gray')
    try:
        os.mkdir('{}/Clusters_binary_vdf_inverted_gauss_paper'.format(output_directory))
    except:
        pass
    plt.savefig('{}/Clusters_binary_vdf_inverted_gauss_paper/Cluster_inverted_gauss_{}'.format(output_directory, counter),bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # log mean DP:
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.imshow(DP_sum.mean(0), norm=colors.LogNorm(), cmap='gray')
    try:
        os.mkdir('{}/Clusters_binary_vdf_mean_paper/'.format(output_directory))
    except:
        pass
    plt.savefig('{}/Clusters_binary_vdf_mean_paper/Cluster_{}'.format(output_directory, counter), norm=colors.LogNorm,bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # Log Max DP in certain cluster:
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.imshow(DP_sum.max(0), norm=colors.LogNorm(), cmap='gray')
    try:
        os.mkdir('{}/Clusters_binary_vdf_max/'.format(output_directory))
    except:
        pass
    plt.savefig('{}/Clusters_binary_vdf_max/Dp_max_{}'.format(output_directory, counter), norm=colors.LogNorm(),bbox_inches='tight', pad_inches=0)
    plt.close()

    # Max DP with inverted gaussian filter:
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.imshow(1-np.exp((-DP_sum.max(0).data)/n_raw), cmap='gray')
    try:
        os.mkdir('{}/Clusters_binary_vdf_max_gauss_paper/'.format(output_directory))
    except:
        pass
    plt.savefig('{}/Clusters_binary_vdf_max_gauss_paper/Dp_max_raw_{}'.format(output_directory, counter), norm=colors.LogNorm(),bbox_inches='tight', pad_inches=0)
    plt.close()
               
    counter +=1

<ipython-input-106-8fcbd057b7b3>:32: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "norm" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig('{}/Clusters_binary_vdf_mean_paper/Cluster_{}'.format(output_directory, counter), norm=colors.LogNorm,bbox_inches='tight', pad_inches=0)
<ipython-input-106-8fcbd057b7b3>:46: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "norm" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig('{}/Clusters_binary_vdf_max/Dp_max_{}'.format(output_directory, counter), norm=colors.LogNorm(),bbox_inches='tight', pad_inches=0)
<ipython-input-106-8fcbd057b7b3>:60: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "norm" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig('{}/Clusters_binary_vdf_max_gauss_paper/Dp_max_raw_{}'.format(output_direct

## Decomposition of the dataset

Do decomposition of the dataset. Any decomposition routine from hyperspy is applicable. Read the hyperspy documentation for more information about the available decomposition routines. 
The goal of the decomposition is to do dimensional reduction and obtain a simplified dataset. The decomposition can be done with or without masks in reciprocal space. It is advisable to use masks at least in reciprocal space, as local variations of the Al reflections will be easily picked up by the decomposition.

In [38]:
components = 6
try:
    signal_decomp
except:
    print('signal_decomp not found, now copying ')
    signal_decomp = signal.deepcopy()
    signal_decomp.change_dtype('float32')
    signal_decomp = px.signals.electron_diffraction2d.ElectronDiffraction2D(signal_decomp)
#vdf_mask = signal.metadata.Masks.VDF.inverted.data # Set to None if you don't want masks
diff_mask = ~circular_mask.data
#diff_mask = signal.metadata.Masks.DP.normal.data 
algorithm = "NMF" # Change here between svd and nmf
max_iter = 200 #200 is default from sklearn

signal_decomp not found, now copying 


In [39]:
signal_decomp.decomposition(algorithm=algorithm, normalize_poissonian_noise=True, output_dimension=components, signal_mask=diff_mask)#, navigation_mask =vdf_mask)

C:\Users\elisathr\AppData\Local\Continuum\anaconda3\envs\pyxem0.13.2\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\elisathr\AppData\Local\Continuum\anaconda3\envs\pyxem0.13.2\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=6
  centre=None
scikit-learn estimator:
NMF(n_components=6)


### Check decomposition

In [40]:
plt.close('all')
%matplotlib qt
signal_decomp.plot_decomposition_results() # Plots the results.

In [ ]:
%matplotlib qt

In [ ]:
# Plot Scree plot from svd decomposition. 
if algorithm == 'svd':
    signal_decomp.plot_explained_variance_ratio()

In [41]:
from ClusterAnalysis import plot as caplt
output_directory = r'E:\PhD\MERLIN\ZM42\Na4d_AA8min\2021_01_22_s10a\20210122 105644\Test_run'

In [42]:
factors = signal_decomp.get_decomposition_factors()
loadings = signal_decomp.get_decomposition_loadings()
caplt.save_component_maps(signal_decomp,factors, loadings, algorithm, output_directory, components, scalebar=False, 
                      saveFactorsLoadings=True) # Scalebar only works if you have matplotlib-scalebar package installed (https://pypi.org/project/matplotlib-scalebar/)

Saving images in new directory:  E:\PhD\MERLIN\ZM42\Na4d_AA8min\2021_01_22_s10a\20210122 105644\Test_runNMF_6_components
Figure dimensions:
	width: 6.944 in
	height:3.472 in.

Figure dimensions:
	width: 6.944 in
	height:3.472 in.

Figure dimensions:
	width: 6.944 in
	height:3.472 in.

Figure dimensions:
	width: 6.944 in
	height:3.472 in.

Figure dimensions:
	width: 6.944 in
	height:3.472 in.

Figure dimensions:
	width: 6.944 in
	height:3.472 in.



### Load pre-decomposed dataset

Load a pre-decomposed dataset and prepare a composite map of components and loadings from the decomposition

In [2]:
import ClusterAnalysis.plot as caplt

In [3]:
path = r'E:\PhD\MERLIN\ZM42\Na4d_AA8min\2021_01_22_s10a\20210122 105644\nmf_6_components'.replace('\\','\\\\')
file_factors = r'/factors_nmf_6comp.hdf5'
file_loadings = r'/loadings_nmf_6comp.hdf5'
factors = hs.load(path + file_factors)
loadings = hs.load(path + file_loadings)

In [4]:
rgb_factors, rgb_loadings, components, rgb_colors = caplt.decomp_as_rgb(factors, loadings, 'rgb')

In [5]:
rgba_factors, rgba_loadings, components, rgba_colors = caplt.decomp_as_rgb(factors, loadings, 'rgba', threshold=0.01)

In [6]:
rgba_loadings.plot()
rgba_factors.plot()

In [7]:
component_start = 2
component_stop = 5
rgb_factors = rgb_factors.inav[component_start:component_stop]
rgb_loadings = rgb_loadings.inav[component_start:component_stop]

rgba_factors = rgba_factors.inav[component_start:component_stop]
rgba_loadings = rgba_loadings.inav[component_start:component_stop]

components = components[component_start:component_stop+1]

In [8]:
import matplotlib
import matplotlib.cm as cm
fontsize = 12
hs.plot.plot_images([rgb_factors.inav[0], rgba_factors, rgba_loadings.inav[0], rgba_loadings], per_row = len(rgba_factors)+1, axes_decor=None, label = None, scalebar=None, 
                scalebar_color='black', padding = {'left': 0, 'right': 1, 'bottom': 0, 'top': 1, 'wspace': 0.01, 'hspace': 0.01})
fig = plt.gcf()
axes = fig.get_axes()
axes[0].cla()
for i in range(len(rgba_factors)):
    rgb_copy = rgba_factors.inav[i].deepcopy()
    rgb_copy.change_dtype('uint16')
    data = rgb_copy.data
    data = (data/np.max(data))
    axes[0].imshow(data)
    
N = len(rgb_factors) + 1
axes[N].cla()
for i in range(len(rgba_loadings)):
    rgb_copy = rgba_loadings.inav[i].deepcopy()
    rgb_copy.change_dtype('uint16')
    data = rgb_copy.data
    data = (data/np.max(data))
    axes[N].imshow(data)

for ax in axes:
    #ax.set_facecolor('k')
    ax.set_xticks([])
    ax.set_yticks([])

fig.set_size_inches((3*len(rgba_factors), 3*2))

C:\Users\elisathr\AppData\Local\Continuum\anaconda3\envs\pyxem0.13.2\lib\site-packages\hyperspy\drawing\utils.py:839: UserWarning: Sorry, colorbar is not implemented for RGB images.
  warnings.warn("Sorry, colorbar is not implemented for RGB images.")
